In [ ]:
#!python -m spacy download en
#nltk.download('stopwords') # run this one time

### Resume tailoring

A company has a requirement for two job roles i.e Android Developer and Journalist. And your manager wants you to do some basic analysis for him.

For most job openings, a particular skill set is desired to perform specific tasks. Tailoring your resume is about recognizing those skills and responsibilities on the job description and making it obvious that you’re up to the task. Your company's goal is to draw the shortest line possible between your experience and what’s stated in the job description.

Tailoring your resume connects the dots for recruiters and hiring managers who are overwhelmed by a flood of generic applicants. Instead of proving that you’re an experienced professional in general, it shows them that you’re a perfect fit for this specific job.

 


### About the dataset
When performing data science tasks, it’s common to use data found on the internet. You’ll usually be able to access this data in CSV format, or via an Application Programming Interface (API). However, there are times when the data you want can only be accessed as part of a web page. In cases like this, you’ll want to use a technique called web scraping to get the data from the web page into a format you can work within your analysis.

You need to perform Topic Modelling on the given data and extract useful topics that will help your manager to short list the candidates based on the topics for a specified job role.

The scraped data is been provided to you in the form of `csv`.

|Feature|Description|
|-----|-----|
|company| Name of the company|
|job| job title|
|job_desc| description of jobs|
|location|job locaton|
|url|Link of the jobs from it was scraped|
|job_type|type of the job|

### Importing necessary libraries

In [5]:
import pandas as pd
import numpy as np
import re
import spacy
import gensim
from gensim import corpora
pd.set_option("display.max_colwidth", 200)


import operator
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.stem import WordNetLemmatizer
from string import punctuation
from nltk.tokenize import word_tokenize
from collections import Counter
import operator

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
%matplotlib inline

### Loading the dataset 

In [6]:
jobs = pd.read_csv("merged_indeed_new.csv")
jobs.head()

,company,job,job_desc,location,url,job_type
0,Micro Focus,Core Java Developer,"Core Java Developer\r\r\n\r\r\nJob Description:\r\r\n\r\r\nAt Micro Focus, everything we do is based on a simple idea: The fastest way to get results is to build on what you have. Our software sol...",Bangalore,https://www.indeed.co.in/pagead/clk?mo=r&ad=-6NYlbfkN0BduEgvIgdT7EDM_O2GxHkw7QoaouEWjxefAvaX3ZwZ9tYBt705y4baMnhcBFo_61Er-rE59t0DIcl816jpSPhQlE2-XsX8ZtBLxXVsMZdq8VWbZfs3uXN1oqCxQ7zxUL2JTHVmAEZthPEL...,Android Developer
1,MS Clinical Research (P) Ltd,Android App Developer,"Job Summary\r\r\nOver 3 years experience designing, developing, integrating, and supporting Android App development\r\r\nApply in-depth understanding of business and IT requirements to streamline ...",Bangalore,https://www.indeed.co.in/pagead/clk?mo=r&ad=-6NYlbfkN0BduEgvIgdT7EDM_O2GxHkw7QoaouEWjxefAvaX3ZwZ9tYBt705y4baMnhcBFo_61Er-rE59t0DIcl816jpSPhQlE2-XsX8ZtBLxXVsMZdq8VWbZfs3uXN1oqCxQ7zxUL2JTHVmAEZthPEL...,Android Developer
2,Applied Materials Inc.,Software Engineer – Unity Developer,"Company Introduction\r\r\nApplied Materials , Inc. is the global leader in materials engineering solutions for the semiconductor, flat panel display and solar photovoltaic (PV) industries. applied...",Bangalore,https://www.indeed.co.in/pagead/clk?mo=r&ad=-6NYlbfkN0BduEgvIgdT7EDM_O2GxHkw7QoaouEWjxefAvaX3ZwZ9tYBt705y4baMnhcBFo_61Er-rE59t0DIcl816jpSPhQlE2-XsX8ZtBLxXVsMZdq8VWbZfs3uXN1oqCxQ7zxUL2JTHVmAEZthPEL...,Android Developer
3,Shaw Academy,Lead Mobile Developer,"Senior Mobile App Developer/Lead (Android or iOS)\r\r\nShaw Academy is seeking a Mobile Development Lead, initially to be the hands-on coder for our apps and then to build a team aroundyou. The ro...",Bangalore,https://www.indeed.co.in/pagead/clk?mo=r&ad=-6NYlbfkN0BduEgvIgdT7EDM_O2GxHkw7QoaouEWjxefAvaX3ZwZ9tYBt705y4baMnhcBFo_61Er-rE59t0DIcl816jpSPhQlE2-XsX8ZtBLxXVsMZdq8VWbZfs3uXN1oqCxQ7zxUL2JTHVmAEZthPEL...,Android Developer
4,Letsgettin Private Limited,Android Developer- Freshers,Job Summary\r\r\nPosition: Android developer\r\r\nEducation: Bachelor's\r\r\nRequired candidates: Freshers\r\r\nResponsibilities and Duties\r\r\nDesign and build advanced applications for the Andr...,Bangalore,https://www.indeed.co.in/pagead/clk?mo=r&ad=-6NYlbfkN0BduEgvIgdT7EDM_O2GxHkw7QoaouEWjxefAvaX3ZwZ9tYBt705y4baMnhcBFo_61Er-rE59t0DIcl816jpSPhQlE2-XsX8ZtBLxXVsMZdq8VWbZfs3uXN1oqCxQ7zxUL2JTHVmAEZthPEL...,Android Developer


### Drop unnecessary Columns

For the analysis of the job description, we are only interested in the text data associated with the jobs. We will analyze this text data using natural language processing. Since the file contains some metadata such as company, location and url. It is necessary to remove all the columns that do not contain useful text information.

In [7]:
# Remove the columns
jobs = jobs.drop(columns=['company', 'job', 'location','url'], axis=1)

# Print out the first rows of papers
jobs.head()

,job_desc,job_type
0,"Core Java Developer\r\r\n\r\r\nJob Description:\r\r\n\r\r\nAt Micro Focus, everything we do is based on a simple idea: The fastest way to get results is to build on what you have. Our software sol...",Android Developer
1,"Job Summary\r\r\nOver 3 years experience designing, developing, integrating, and supporting Android App development\r\r\nApply in-depth understanding of business and IT requirements to streamline ...",Android Developer
2,"Company Introduction\r\r\nApplied Materials , Inc. is the global leader in materials engineering solutions for the semiconductor, flat panel display and solar photovoltaic (PV) industries. applied...",Android Developer
3,"Senior Mobile App Developer/Lead (Android or iOS)\r\r\nShaw Academy is seeking a Mobile Development Lead, initially to be the hands-on coder for our apps and then to build a team aroundyou. The ro...",Android Developer
4,Job Summary\r\r\nPosition: Android developer\r\r\nEducation: Bachelor's\r\r\nRequired candidates: Freshers\r\r\nResponsibilities and Duties\r\r\nDesign and build advanced applications for the Andr...,Android Developer


###  Calculate the number of jobs for each job type. 

In [ ]:
jobs["job_type"].value_counts()

### Subset the jobs(i.e Android Developer & Journalist) based on job_type. And store only job_desc based on job type.

Further analysis will be done only on `job_desc` column.

In [ ]:
job_requirement  = jobs[(jobs["job_type"] == "Android Developer") | (jobs["job_type"] == "Journalist")][['job_desc']]
job_requirement.head()

### Retain alphabets/remove unnecessary space
Now, we will perform some simple preprocessing on the job description column(i.e `job_desc`) in order to make them more amenable for analysis. We will use a regular expression to retain only alphabets in the description and remove unnecessary space.

In [ ]:
# Retain only alphabets
job_requirement['job_desc'] = job_requirement['job_desc'].apply(lambda x:re.sub('[^a-zA-Z]', ' ',x))

# Remove unnecesary space
job_requirement['job_desc'] = job_requirement['job_desc'].apply(lambda x:re.sub(' +', ' ',x))

### Exploratory Analysis: Plot the word cloud of the most common words
In order to verify whether the preprocessing happened correctly, we can make a word cloud of the text of the job descriptions. This will give us a visual representation of the most common words. Visualization is key to understanding whether we are still on the right track! In addition, it allows us to verify whether we need additional preprocessing before further analyzing the text data. Python has a massive number of open libraries! Instead of trying to develop a method to create word clouds ourselves, we'll use Andreas Mueller's wordcloud library

In [ ]:

# Import the wordcloud library
from wordcloud import WordCloud

# Join the different processed jobs together.
long_string = ','.join(list(job_requirement['job_desc'].values))

# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=5000, contour_width=3, contour_color='steelblue')

# Generate a word cloud
wordcloud.generate(long_string)

# Visualize the word cloud
wordcloud.to_image()

### Observation:

- In above wordcloud we see the most common words that occur in the job descriptions are Job,skill, year,etc. So we need to remove these words as to get the useful words related to the job. 

### Let's remove some common words that every job description contain. A common words list is provided to you(you can add more). Display top 10 most occuring words.
LDA does not work directly on text data. First, it is necessary to convert the documents into a simple vector representation. This representation will then be used by LDA to determine the topics. Each entry of a 'document vector' will correspond with the number of times a word occurred in the document.

In [ ]:
# List of some common words 
stop_set = ['experience','year','years','job','work','knowledge','skills','preferred','required','responsibilities','education','salary','ability']

#  removing stop words
stop = set(stopwords.words('english'))
for word in stop_set:
    stop.add(word)

custom = list(stop)+list(punctuation)

# Initialize CountVectorizer
vectorizer = CountVectorizer(stop_words=custom , max_features=30000)

# Transform job description
description = vectorizer.fit_transform(job_requirement['job_desc'])

# initialize empty dictionary
words = {}

# initialize with 0
i = 0

# Number of time every feature appears over the entire document
sums = np.array(np.sum(description, axis=0)).flatten()

# Loop to map 'sums' to its word
for word in vectorizer.get_feature_names():
    words[word] = sums[i]
    i += 1

# Top 20 most occuring words
top_20 = sorted(words.items(), key=operator.itemgetter(1), reverse=True)[:20]

# Display top 20 words
top_20_words = [i[0] for i in top_20]
top_20_values = [i[1] for i in top_20]
plt.figure(figsize=(20,10))
sns.barplot(top_20_words,top_20_values)
plt.show()

It seems that now most frequent terms in our data are relevant.

### Perform lowercasing and calculate the frequency of top 10  words. 

In [ ]:
# custom stopword list is provided to you in upper task

# Code starts here

# # Converting the rows of column 'job_desc' into a bag of words
BoW = job_requirement["job_desc"].str.lower().tolist()

# Tokenizing the words
BoW = [word_tokenize(x) for x in BoW]

# Flattening the list
BoW = [item for sublist in BoW for item in sublist]

# Removing the stopwords
BoW = [x for x in BoW if x not in custom]

# Initializing wordnet lemmatizer
lemmatizer = WordNetLemmatizer()

# Lemmatizing the bag of words
BoW = [lemmatizer.lemmatize(x) for x in BoW]

# Create a dictionary from the counter
BoW_dict = dict(Counter(BoW))

# Sort the words
sorted_d = sorted(BoW_dict.items(), key=operator.itemgetter(1), reverse=True)

print("\nTop 10 words:\n", sorted_d[:10])

# Code ends here

### Model buliding with LSA

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
from gensim.models.lsimodel import LsiModel
from gensim import corpora
from pprint import pprint


exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

# Function to lemmatize and remove the stopwords
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in custom])
    punc_free = "".join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

# Creating a list of documents from the complaints column
list_of_docs = job_requirement["job_desc"].tolist()

# Implementing the function for all the complaints of list_of_docs
doc_clean = [clean(doc).split() for doc in list_of_docs]
print(doc_clean[:2])

# Code starts here

# Creating the dictionary id2word from our cleaned word list doc_clean
dictionary = corpora.Dictionary(doc_clean)

# Creating the corpus
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

# Creating the LSi model
lsimodel = LsiModel(corpus=doc_term_matrix, num_topics=2, id2word=dictionary)
pprint(lsimodel.print_topics())



#### Observation:
- The first topic Topic 0 has terms like ‘android’, ‘development’, ‘mobile’, ‘technology’, indicating that the topic is very much related to Android developer job.
- Topic 1 has terms like ‘medium’, ‘communication’, ‘marketing’, ‘journalist’, indicating that the topic is very much related to Journalist job but we got the term android also which is no were related to Journalist job.

### Model buliding with LDA

In LSA we saw that Topic 1 has some different words which are not related to the Journalist job. Lets see if we can improve our topics by using LDA algorithm.

In [ ]:
# imports
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel

#doc_clean - Cleaned list containing entries of 'job_desc' column of our dataframe
# This variable is already available.

# Code starts here

# Creating the dictionary id2word from our cleaned word list doc_clean
dictionary = corpora.Dictionary(doc_clean)

# Creating the corpus
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]


# Creating the LDA model
ldamodel = LdaMulticore(corpus=doc_term_matrix, num_topics=2,id2word=dictionary, random_state=20, passes=30, workers= 12)

# printing the topics
pprint(ldamodel.print_topics())

#### Observation:
The first topic Topic 0 has terms like ‘medium’, ‘communication’, ‘marketing’, ‘journalist’, indicating that the topic is very much related to Journalist job. Similarly, Topic 1 seems to be about the Android development job as it has terms like ‘android’, ‘development’,‘mobile’,‘technology’, and ‘design’.

### Analyzing with LDA model

To visualize the topics in a 2-dimensional space we will use the pyLDAvis library. This visualization is interactive in nature and displays topics along with the most relevant words.

pyLDAvis package is designed to help users interpret the topics in a topic model that has been fit to a corpus of text data. The interactive visualization pyLDAvis produces is helpful for both:

1. Better understanding and interpreting individual topics, and
2. Better understanding the relationships between the topics.

For (1), you can manually select each topic to view its top most freqeuent and/or “relevant” terms, using different values of the λ parameter. This can help when you’re trying to assign a human interpretable name or “meaning” to each topic.

For (2), exploring the Intertopic Distance Plot can help you learn about how topics relate to each other, including potential higher-level structure between groups of topics.


Here is the documentation for <a href="https://pyldavis.readthedocs.io/en/latest/readme.html">pyLDAvis</a>

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)
vis